In [1]:
import pandas
import mmengine
# info = pandas.read_pickle('data/kitti-360/kitti360_infos_all.pkl')
from projects.detr3d.custom_concat_dataset import CustomNusc
from projects.configs.mono_ablation import cfg_for_filter_object_by_range as cfg
import copy
from mmdet3d.utils import register_all_modules, replace_ceph_backend
from projects.detr3d.vis_zlt import visualizer_zlt
from projects.detr3d.detr3d import DETR3D
import os.path as osp
import numpy as np
from projects.detr3d.detr3d import DETR3D
import torch
from mmdet3d.structures import LiDARInstance3DBoxes
from mmengine.structures import InstanceData
from mmdet3d.registry import DATASETS
detr3d = DETR3D()
vis = visualizer_zlt(debug_name='',vis_count=10000)
register_all_modules()

In [3]:
nusc = DATASETS.build(cfg.nusc_train)
CC = cfg.waymo_train
# CC['ann_file']='debug_val_mono_front.pkl'
# CC['data_root'] = 'data/waymo_dev1x/kitti_format/'
waymo = DATASETS.build(CC)
lyft = DATASETS.build(cfg.lyft_train)

06/09 03:58:35 - mmengine - INFO - ------------------------------
06/09 03:58:35 - mmengine - INFO - The length of the dataset: 358
06/09 03:58:35 - mmengine - INFO - The number of instances per category in the dataset:
+------------+--------+
| category   | number |
+------------+--------+
| Car        | 2608   |
| Pedestrian | 1263   |
| Cyclist    | 33     |
+------------+--------+


In [6]:
import shutil
for i in range(len(waymo)):
    fname = waymo[i]['data_samples'].metainfo['img_path']
    shutil.copy(fname,'debug/waymo_raw/'+osp.basename(fname))

TypeError: 'int' object is not iterable

In [23]:
kitti = DATASETS.build(cfg.kitti_train)
K360 = DATASETS.build(cfg.K360_train)
argo2 = DATASETS.build(cfg.argo2_train)

06/02 15:50:26 - mmengine - INFO - ------------------------------
06/02 15:50:26 - mmengine - INFO - The length of the dataset: 3712
06/02 15:50:26 - mmengine - INFO - The number of instances per category in the dataset:
+----------------+--------+
| category       | number |
+----------------+--------+
| Pedestrian     | 2207   |
| Cyclist        | 734    |
| Car            | 14357  |
| Van            | 1297   |
| Truck          | 488    |
| Person_sitting | 56     |
| Tram           | 224    |
| Misc           | 337    |
+----------------+--------+
06/02 15:51:31 - mmengine - INFO - ------------------------------
06/02 15:51:31 - mmengine - INFO - The length of the dataset: 49253
06/02 15:51:31 - mmengine - INFO - The number of instances per category in the dataset:
+----------------------+--------+
| category             | number |
+----------------------+--------+
| bicycle              | 29247  |
| box                  | 0      |
| bridge               | 0      |
| building       

In [5]:
def fake_loadimg(results: dict):
    filename, cam2img, lidar2cam = [], [], []
    for _, cam_item in results['images'].items():
        if 'img_path' not in cam_item: # for kitti-mono
            continue
        filename.append(cam_item['img_path'])
        cam2img.append(cam_item['cam2img'])
        lidar2cam.append(cam_item['lidar2cam'])

    results['filename'] = filename
    results['img_path'] = results['filename']
    results['cam2img'] = cam2img
    results['lidar2cam'] = lidar2cam

    results['ori_cam2img'] = copy.deepcopy(results['cam2img'])

    results['filename'] = filename
    return results
def filter_dim(dim, dimrange):
    mask = dim[:,0]>-10
    for i in range(3):
        mask &= (dimrange[i]<=dim[:,i])&(dim[:,i]<=dimrange[i+3])
    return mask


In [32]:
kittival = DATASETS.build(cfg.kitti_val)

06/02 16:04:40 - mmengine - INFO - ------------------------------
06/02 16:04:40 - mmengine - INFO - The length of the dataset: 3769
06/02 16:04:40 - mmengine - INFO - The number of instances per category in the dataset:
+----------------+--------+
| category       | number |
+----------------+--------+
| Pedestrian     | 2280   |
| Cyclist        | 893    |
| Car            | 14385  |
| Van            | 1617   |
| Truck          | 606    |
| Person_sitting | 166    |
| Tram           | 287    |
| Misc           | 636    |
+----------------+--------+


In [48]:
datasets = [nusc,waymo,lyft,argo2,K360]
from mmdet3d.datasets.transforms.transforms_3d import ObjectRangeFilter
bevrange = [28,-5,30,5]
dimrange = [2,1.6,1.75,5,2.3,1.85]
def get_corl_images(nusc):
    lst = []
    vis = visualizer_zlt('debug/CoRL',debug_name='fig1')
    vis2 = visualizer_zlt('debug/CoRL',debug_name='origin',vis_count=0)
    for i in range(0,len(nusc)):
        info = nusc.get_data_info(i)
        info = fake_loadimg(info)
        info = nusc.pipeline(info)
        bbox = info['data_samples'].gt_instances_3d['bboxes_3d']
        label = info['data_samples'].gt_instances_3d['labels_3d']
        mask = bbox.in_range_bev(bevrange) & (label==0)
        mask &= (bbox.tensor[:,-1] > 3.0) | (bbox.tensor[:,-1] < 0.1)
        mask = mask & (bbox.tensor[:,-1]<0.1) & (bbox.tensor[:,-1]>-0.1)
        mask = mask & (bbox.tensor[:,5]<1.9) & (bbox.tensor[:,5]>1.7)
        mask = mask & (bbox.tensor[:,4]<2.1) & (bbox.tensor[:,4]>1.9)
        mask = mask & (bbox.tensor[:,3]>4.7)
        # mask &= filter_dim(dim, dimrange)
        # bbox = bbox[mask]
        if mask.any():
            print(bbox[mask])
            print(info['data_samples'].metainfo['lidar_path'])
            meta = info['data_samples'].metainfo
            # argo2 fails to read point cloud due to unknown lidar_path substitution
            # it won't occur if we use original config
            vis.visualize(info['data_samples'].gt_instances_3d[mask],meta)
            vis2.visualize(info['data_samples'].gt_instances_3d[mask],meta)
            lst.append(info['data_samples'].metainfo['img_path'])
            if len(lst)>20:
                break
    print(lst)

for ds in [argo2]:
    get_corl_images(ds)

LiDARInstance3DBoxes(
    tensor([[ 2.8640e+01, -4.7373e+00, -4.8628e-01,  5.6518e+00,  1.9967e+00,
          1.8461e+00, -6.2871e-03]]))
/localdata_ssd/argo2_dev1x/train/108d2060-8bef-3d1c-88c5-c8295f596595/sensors/lidar/315969502460016000.feather
DTW


FileNotFoundError: [Errno 2] Failed to open local file '/localdata_ssd/argo2_dev1x/train/108d2060-8bef-3d1c-88c5-c8295f596595/calibration/egovehicle_SE3_sensor.feather'. Detail: [errno 2] No such file or directory

In [47]:
# datasets = [nusc,waymo,lyft,kitti,argo2,K360]
from mmdet3d.datasets.transforms.transforms_3d import ObjectRangeFilter
bevrange = [25,-5,35,5]
dimrange = [2,1.6,1.75,5,2.3,1.85]
def get_corl_images(nusc):
    lst = []
    vis = visualizer_zlt('debug/CoRL',debug_name='fig1')
    vis2 = visualizer_zlt('debug/CoRL',debug_name='origin',vis_count=0)
    for i in range(0,len(nusc)):
        info = nusc.get_data_info(i)
        info = fake_loadimg(info)
        info = nusc.pipeline(info)
        bbox = info['data_samples'].gt_instances_3d['bboxes_3d']
        label = info['data_samples'].gt_instances_3d['labels_3d']
        mask = bbox.in_range_bev(bevrange) & (label==0)
        # mask &= (bbox.tensor[:,-1] > 3.0) | (bbox.tensor[:,-1] < 0.1)
        mask = mask & (bbox.tensor[:,-1]<0.1) & (bbox.tensor[:,-1]>-0.1)
        mask = mask & (bbox.tensor[:,5]<2.0) & (bbox.tensor[:,5]>1.8)
        mask = mask & (bbox.tensor[:,4]<2.1) & (bbox.tensor[:,4]>1.8)
        # mask = mask & (bbox.tensor[:,3]>4.7)
        # mask &= filter_dim(dim, dimrange)
        # bbox = bbox[mask]
        if mask.any():
            print(bbox[mask])
            vis.visualize(info['data_samples'].gt_instances_3d[mask],info['data_samples'].metainfo)
            vis2.visualize(info['data_samples'].gt_instances_3d[mask],info['data_samples'].metainfo)
            lst.append(info['data_samples'].metainfo['img_path'])
            if len(lst)>20:
                break
    print(lst)

for ds in [kitti,kittival]:
    get_corl_images(ds)

[]
LiDARInstance3DBoxes(
    tensor([[ 3.4698e+01, -4.1514e+00, -1.2765e+00,  4.6300e+00,  2.0400e+00,
          1.9100e+00, -7.9632e-04]]))
Germany
reference torch.Size([2, 3])
375 1242
torch.Size([1, 1, 3])
frame: 601
Germany
reference torch.Size([1, 3])
375 1242
torch.Size([1, 0, 3])
frame: 601
LiDARInstance3DBoxes(
    tensor([[ 3.3048e+01, -4.1318e+00, -1.2736e+00,  4.6300e+00,  2.0400e+00,
          1.9100e+00,  9.2037e-03]]))
Germany
reference torch.Size([2, 3])
375 1242
torch.Size([1, 1, 3])
frame: 976
Germany
reference torch.Size([1, 3])
375 1242
torch.Size([1, 0, 3])
frame: 976
LiDARInstance3DBoxes(
    tensor([[ 3.0269e+01, -2.5710e+00, -1.4061e+00,  4.4100e+00,  1.8200e+00,
          1.9700e+00, -1.0796e-02]]))
Germany
reference torch.Size([2, 3])
375 1242
torch.Size([1, 1, 3])
frame: 1055
Germany
reference torch.Size([1, 3])
375 1242
torch.Size([1, 0, 3])
frame: 1055
LiDARInstance3DBoxes(
    tensor([[ 2.6609e+01, -2.7113e+00, -1.4559e+00,  4.4100e+00,  1.8200e+00,
       

In [ ]:
waymoval = DATASETS.build(cfg.waymo_val)

In [20]:
vis = visualizer_zlt('debug/CoRL',debug_name='fig1')
vis2 = visualizer_zlt('debug/CoRL',debug_name='origin',vis_count=0)
info = waymoval.get_data_info(703)
info = fake_loadimg(info)
info = waymoval.pipeline(info)
print(info['data_samples'].gt_instances_3d)
# mask = (info['data_samples'].gt_instances_3d['labels_3d'] == 0)
# # /localdata_ssd/waymo_dev1x/training/image_0/1056194.jpg
# vis.visualize(info['data_samples'].gt_instances_3d[mask],info['data_samples'].metainfo)
# vis2.visualize(info['data_samples'].gt_instances_3d[mask],info['data_samples'].metainfo)

<InstanceData(

    META INFORMATION

    DATA FIELDS
    labels_3d: tensor([0])
    bboxes_3d: LiDARInstance3DBoxes(
            tensor([[ 2.8936e+01,  3.9331e+00, -5.4543e-03,  5.0000e+00,  2.2000e+00,
                  1.8000e+00, -7.9632e-04]]))
) at 0x7fa679f18250>


In [39]:
info = waymo.get_data_info(16454)
info = fake_loadimg(info)
info = waymo.pipeline(info)
print(info['data_samples'].gt_instances_3d)# 4.1300e+00,  1.9500e+00 1.8100e+00

info = waymo.get_data_info(17440)
info = fake_loadimg(info)
info = waymo.pipeline(info)
print(info['data_samples'].gt_instances_3d)# 4.9100e+00, 2.0800e+00, 1.7400e+00

info = kitti.get_data_info(3502) # kitti 不行
info = fake_loadimg(info)
info = kitti.pipeline(info)
print(info['data_samples'].gt_instances_3d)# 4.1100,  1.6000,  1.7300

info = kitti.get_data_info(43) # kitti 不行
info = fake_loadimg(info)
info = kitti.pipeline(info)
print(info['data_samples'].gt_instances_3d)# 4.5000e+00,  1.6300e+00 1.6200e+00

info = kittival.get_data_info(3165)
info = fake_loadimg(info)
info = kittival.pipeline(info)
print(info['data_samples'].gt_instances_3d)# 4.4100e+00,  1.8200e+00 1.9700e+00


info = K360.get_data_info(8556)
info = fake_loadimg(info)
info = K360.pipeline(info)
print(info['data_samples'].gt_instances_3d)# 5.0172e+00,  2.0444e+00 1.8295e+00

info = K360.get_data_info(29220)
info = fake_loadimg(info)
info = K360.pipeline(info)
print(info['data_samples'].gt_instances_3d) # 5.1060e+00,  2.0241e+00 1.7090e+00

info = K360.get_data_info(44366)
info = fake_loadimg(info)
info = K360.pipeline(info)
print(info['data_samples'].gt_instances_3d) # 4.9465e+00,  2.0476e+00, 1.8931e+00

info = lyft.get_data_info(5162)
info = fake_loadimg(info)
info = lyft.pipeline(info)
print(info['data_samples'].gt_instances_3d)# 4.7150e+00,  1.9760e+00, 1.8250e+00

info = nusc.get_data_info(7012)
info = fake_loadimg(info)
info = nusc.pipeline(info)
print(info['data_samples'].gt_instances_3d) # 4.7800e+00,  1.9820e+00 1.8450e+00

info = argo2.get_data_info(10831)
info = fake_loadimg(info)
info = argo2.pipeline(info)
print(info['data_samples'].gt_instances_3d)# 5.2694e+00,  1.9725e+00, 1.8815e+00

<InstanceData(

    META INFORMATION

    DATA FIELDS
    labels_3d: tensor([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])
    bboxes_3d: LiDARInstance3DBoxes(
            tensor([[ 4.2257e+01,  1.2506e+01,  6.0629e-01,  6.8000e-01,  7.7000e-01,
                  1.3100e+00, -1.5408e+00],
                [ 3.5085e+01,  1.3053e+01,  7.2417e-01,  6.4000e-01,  5.9000e-01,
                  1.8200e+00, -5.7080e-01],
                [ 2.9713e+01, -1.8637e-01, -1.5216e-01,  4.1300e+00,  1.9500e+00,
                  1.8100e+00, -7.9632e-04],
                [ 2.6434e+01, -7.8276e+00,  2.3397e-01,  7.4000e-01,  9.7000e-01,
                  1.7400e+00,  1.6692e+00],
                [ 3.5684e+01,  1.3107e+01,  7.0635e-01,  7.4000e-01,  7.3000e-01,
                  1.7200e+00, -1.5408e+00],
                [ 4.6349e+01,  1.2441e+01,  6.9136e-01,  6.4000e-01,  6.0000e-01,
                  1.6600e+00, -2.5808e+00],
                [ 2.9128e+01, -7.0211e+00,  1.4087e-01,  8.4000e-01,  7.3000e